<a href="https://colab.research.google.com/github/mfmceneaney/DIRC_COLAB/blob/master/DIRC_CNN_Architecture_Evaluation_V10_MASTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Using GPU speeds up CNN training
# Go to Edit > Notebook Settings > Hardware Accelerator and select GPU (TPU is very slow)

In [0]:
print(" > Installing uproot...")
!pip install uproot

print(' > Installing h5py...')
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [0]:
import uproot

from os import chdir, getcwd, mkdir, listdir, rmdir
from shutil import rmtree

# Standard scientific Python imports
import matplotlib.pyplot as plt
import numpy as np
import math

# Graphics Imports
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Image transformations # Check out https://scikit-image.org/docs/dev/api/skimage.transform.html#skimage.transform.resize
from skimage.transform import rescale, resize, downscale_local_mean

from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x 
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, metrics, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121, ResNet50V2, VGG16, VGG19, MobileNet, MobileNetV2, NASNetLarge, NASNetMobile
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop, SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy

# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
import datetime

# Clear any logs from previous runs
# !rm -rf ./logs/ 

# Imports for custom callbacks
from tensorflow.keras.callbacks import Callback, ProgbarLogger
from tensorflow.keras.callbacks import ModelCheckpoint
#### Check out this link for tf.keras vs. tf.python.keras : https://github.com/tensorflow/tensorflow/issues/33075#issuecomment-539070546

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
chdir('/content/gdrive/My Drive/DIRC_CNN/') # This directory created 1/22/20 18:33:00
listdir()
chdir('test') # Store files in the test directory temporarily in case something goes wrong
listdir()


In [0]:
# Get data from files
print("Opening files")

# 3GeV
piplus3t4 = uproot.open("/content/gdrive/My Drive/piplus_p3_theta4_flat.root")["dircml_flat"]
kplus3t4 = uproot.open("/content/gdrive/My Drive/kplus_p3_theta4_flat.root")["dircml_flat"]
# mkdir('p3_theta4')
# chdir('p3_theta4')

# 4GeV
piplus4t4 = uproot.open("/content/gdrive/My Drive/piplus_p4_theta4_flat.root")["dircml_flat"]
kplus4t4 = uproot.open("/content/gdrive/My Drive/kplus_p4_theta4_flat.root")["dircml_flat"]
# mkdir('p4_theta4')
# chdir('p4_theta4')

# 5GeV
piplus5t4 = uproot.open("/content/gdrive/My Drive/piplus_p5_theta4_flat.root")["dircml_flat"]
kplus5t4 = uproot.open("/content/gdrive/My Drive/kplus_p5_theta4_flat.root")["dircml_flat"]
# mkdir('p5_theta4')
# chdir('p5_theta4')

# 6GeV
piplus6t4 = uproot.open("/content/gdrive/My Drive/piplus_p6_theta4_flat.root")["dircml_flat"]
kplus6t4 = uproot.open("/content/gdrive/My Drive/kplus_p6_theta4_flat.root")["dircml_flat"]
# mkdir('p6_theta4')
# chdir('p6_theta4')

# 7GeV
piplus7t4 = uproot.open("/content/gdrive/My Drive/tree_piplus_p7_theta4_flat.root")["dircml_flat"]
kplus7t4 = uproot.open("/content/gdrive/My Drive/tree_kplus_p7_theta4_flat.root")["dircml_flat"]
# mkdir('p7_theta4')
# chdir('p7_theta4')

# 8GeV
piplus8t4 = uproot.open("/content/gdrive/My Drive/tree_piplus_p8_theta4_flat.root")["dircml_flat"]
kplus8t4 = uproot.open("/content/gdrive/My Drive/tree_kplus_p8_theta4_flat.root")["dircml_flat"]
# mkdir('p8_theta4')
# chdir('p8_theta4')

# 9GeV
piplus9t4 = uproot.open("/content/gdrive/My Drive/tree_piplus_p9_theta4_flat.root")["dircml_flat"]
kplus9t4 = uproot.open("/content/gdrive/My Drive/tree_kplus_p9_theta4_flat.root")["dircml_flat"]
# mkdir('p9_theta4')
# chdir('p9_theta4')

# 10GeV
piplus10t4 = uproot.open("/content/gdrive/My Drive/tree_piplus_p10_theta4_flat.root")["dircml_flat"]
kplus10t4 = uproot.open("/content/gdrive/My Drive/tree_kplus_p10_theta4_flat.root")["dircml_flat"]
# mkdir('p10_theta4')
# chdir('p10_theta4')

# 11GeV
piplus11t4 = uproot.open("/content/gdrive/My Drive/tree_piplus_p11_theta4_flat.root")["dircml_flat"]
kplus11t4 = uproot.open("/content/gdrive/My Drive/tree_kplus_p11_theta4_flat.root")["dircml_flat"]
# mkdir('p11_theta4')
# chdir('p11_theta4')

################################## PHI -45 ##################################
# # 3GeV
# mom = '3GeV'
piplus3t4phim45 = uproot.open("/content/gdrive/My Drive/piplus_p3_theta4_phim45_flat.root")["dircml_flat"]
kplus3t4phim45 = uproot.open("/content/gdrive/My Drive/kplus_p3_theta4_phim45_flat.root")["dircml_flat"]
# # # rmtree('p3_theta4_phim45')
# # mkdir('p3_theta4_phim45')
# chdir('p3_theta4_phim45')
# listdir()

# 4GeV
# mom = '4GeV'
piplus4t4phim45 = uproot.open("/content/gdrive/My Drive/piplus_p4_theta4_phim45_flat.root")["dircml_flat"]
kplus4t4phim45 = uproot.open("/content/gdrive/My Drive/kplus_p4_theta4_phim45_flat.root")["dircml_flat"]
# mkdir('p4_theta4_phim45')
# chdir('p4_theta4_phim45')
# listdir()

# 5GeV
# mom = '5GeV'
piplus5t4phim45 = uproot.open("/content/gdrive/My Drive/piplus_p5_theta4_phim45_flat.root")["dircml_flat"]
kplus5t4phim45 = uproot.open("/content/gdrive/My Drive/kplus_p5_theta4_phim45_flat.root")["dircml_flat"]
# # mkdir('p5_theta4_phim45')
# chdir('p5_theta4_phim45')

# # 6GeV
# mom = '6GeV'
piplus6t4phim45 = uproot.open("/content/gdrive/My Drive/piplus_p6_theta4_phim45_flat.root")["dircml_flat"]
kplus6t4phim45 = uproot.open("/content/gdrive/My Drive/kplus_p6_theta4_phim45_flat.root")["dircml_flat"]
# mkdir('p6_theta4_phim45')
# chdir('p6_theta4_phim45')

# # 7GeV
# mom = '7GeV'
piplus7t4phim45 = uproot.open("/content/gdrive/My Drive/piplus_p7_theta4_phim45_flat.root")["dircml_flat"]
kplus7t4phim45 = uproot.open("/content/gdrive/My Drive/kplus_p7_theta4_phim45_flat.root")["dircml_flat"]
# # mkdir('p7_theta4_phim45')
# chdir('p7_theta4_phim45')

# # 8GeV
# mom = '8GeV'
piplus8t4phim45 = uproot.open("/content/gdrive/My Drive/piplus_p8_theta4_phim45_flat.root")["dircml_flat"]
kplus8t4phim45 = uproot.open("/content/gdrive/My Drive/kplus_p8_theta4_phim45_flat.root")["dircml_flat"]
# mkdir('p8_theta4_phim45')
# chdir('p8_theta4_phim45')

# # 9GeV
# mom = '9GeV'
piplus9t4phim45 = uproot.open("/content/gdrive/My Drive/piplus_p9_theta4_phim45_flat.root")["dircml_flat"]
kplus9t4phim45 = uproot.open("/content/gdrive/My Drive/kplus_p9_theta4_phim45_flat.root")["dircml_flat"]
# mkdir('p9_theta4_phim45')
# chdir('p9_theta4_phim45')
# listdir()

# # 10GeV
# mom = '10GeV'
piplus10t4phim45 = uproot.open("/content/gdrive/My Drive/piplus_p10_theta4_phim45_flat.root")["dircml_flat"]
kplus10t4phim45 = uproot.open("/content/gdrive/My Drive/kplus_p10_theta4_phim45_flat.root")["dircml_flat"]
# mkdir('p10_theta4_phim45')
# chdir('p10_theta4_phim45')

# # 11GeV
# mom = '11GeV'
piplus11t4phim45 = uproot.open("/content/gdrive/My Drive/piplus_p11_theta4_phim45_flat.root")["dircml_flat"]
kplus11t4phim45 = uproot.open("/content/gdrive/My Drive/kplus_p11_theta4_phim45_flat.root")["dircml_flat"]
# mkdir('p11_theta4_phim45')
# chdir('p11_theta4_phim45')


################################## THETA 8 ##################################
# # 3GeV
# mom = '3GeV'
piplus3t8 = uproot.open("/content/gdrive/My Drive/piplus_p3_theta8_flat.root")["dircml_flat"]
kplus3t8 = uproot.open("/content/gdrive/My Drive/kplus_p3_theta8_flat.root")["dircml_flat"]
# # mkdir('p3_theta8')
# chdir('p3_theta8')

# # 4GeV
# mom = '4GeV'
piplus4t8 = uproot.open("/content/gdrive/My Drive/piplus_p4_theta8_flat.root")["dircml_flat"]
kplus4t8 = uproot.open("/content/gdrive/My Drive/kplus_p4_theta8_flat.root")["dircml_flat"]
# # mkdir('p4_theta8')
# chdir('p4_theta8')
# listdir()

# # 5GeV
# mom = '5GeV'
piplus5t8 = uproot.open("/content/gdrive/My Drive/piplus_p5_theta8_flat.root")["dircml_flat"]
kplus5t8 = uproot.open("/content/gdrive/My Drive/kplus_p5_theta8_flat.root")["dircml_flat"]
# # mkdir('p5_theta8')
# chdir('p5_theta8')

# # 6GeV
# mom = '6GeV'
piplus6t8 = uproot.open("/content/gdrive/My Drive/piplus_p6_theta8_flat.root")["dircml_flat"]
kplus6t8 = uproot.open("/content/gdrive/My Drive/kplus_p6_theta8_flat.root")["dircml_flat"]
# # mkdir('p6_theta8')
# chdir('p6_theta8')

# # 7GeV
# mom = '7GeV'
piplus7t8 = uproot.open("/content/gdrive/My Drive/piplus_p7_theta8_flat.root")["dircml_flat"]
kplus7t8 = uproot.open("/content/gdrive/My Drive/kplus_p7_theta8_flat.root")["dircml_flat"]
# # mkdir('p7_theta8')
# chdir('p7_theta8')

# # 8GeV
# mom = '8GeV'
piplus8t8 = uproot.open("/content/gdrive/My Drive/piplus_p8_theta8_flat.root")["dircml_flat"]
kplus8t8 = uproot.open("/content/gdrive/My Drive/kplus_p8_theta8_flat.root")["dircml_flat"]
# # mkdir('p8_theta8')
# chdir('p8_theta8')

# # 9GeV
# mom = '9GeV'
piplus9t8 = uproot.open("/content/gdrive/My Drive/piplus_p9_theta8_flat.root")["dircml_flat"]
kplus9t8 = uproot.open("/content/gdrive/My Drive/kplus_p9_theta8_flat.root")["dircml_flat"]
# # mkdir('p9_theta8')
# chdir('p9_theta8')
# listdir()

# # 10GeV
# mom = '10GeV'
piplus10t8 = uproot.open("/content/gdrive/My Drive/piplus_p10_theta8_flat.root")["dircml_flat"]
kplus10t8 = uproot.open("/content/gdrive/My Drive/kplus_p10_theta8_flat.root")["dircml_flat"]
# mkdir('cross_check')
# chdir('cross_check')

# # 11GeV
# mom = '11GeV'
piplus11t8 = uproot.open("/content/gdrive/My Drive/piplus_p11_theta8_flat.root")["dircml_flat"]
kplus11t8 = uproot.open("/content/gdrive/My Drive/kplus_p11_theta8_flat.root")["dircml_flat"]
# mkdir('p11_theta8')
# chdir('p11_theta8')


data = [[[kplus3t4,piplus3t4],
        [kplus4t4,piplus4t4],
        [kplus5t4,piplus5t4],
        [kplus6t4,piplus6t4],
        [kplus7t4,piplus7t4],
        [kplus8t4,piplus8t4],
        [kplus9t4,piplus9t4],
        [kplus10t4,piplus10t4],
        [kplus11t4,piplus11t4]],
        [[kplus3t4phim45,piplus3t4phim45],
        [kplus4t4phim45,piplus4t4phim45],
        [kplus5t4phim45,piplus5t4phim45],
        [kplus6t4phim45,piplus6t4phim45],
        [kplus7t4phim45,piplus7t4phim45],
        [kplus8t4phim45,piplus8t4phim45],
        [kplus9t4phim45,piplus9t4phim45],
        [kplus10t4phim45,piplus10t4phim45],
        [kplus11t4phim45,piplus11t4phim45]],
        [[kplus3t8,piplus3t8],
        [kplus4t8,piplus4t8],
        [kplus5t8,piplus5t8],
        [kplus6t8,piplus6t8],
        [kplus7t8,piplus7t8],
        [kplus8t8,piplus8t8],
        [kplus9t8,piplus9t8],
        [kplus10t8,piplus10t8],
        [kplus11t8,piplus11t8]]]

In [0]:
# mkdir('4_18_p3to6_ALL_ANGLES_ORIGINAL_SPLIT_CHANNELS_METHOD')
# chdir('4_18_p3to6_ALL_ANGLES_ORIGINAL_SPLIT_CHANNELS_METHOD')
# mkdir('4_18_p5_theta4_MODELS_COMPARISONS_ORIGINAL_SPLIT_CHANNELS_METHOD')
# chdir('4_18_p5_theta4_MODELS_COMPARISONS_ORIGINAL_SPLIT_CHANNELS_METHOD')
# mkdir('4_18_p6_theta4_MODELS_COMPARISONS_ORIGINAL_SPLIT_CHANNELS_METHOD')
chdir('4_18_p6_theta4_MODELS_COMPARISONS_ORIGINAL_SPLIT_CHANNELS_METHOD')
listdir()

# Transfer data to appropriately shaped np arrays
images_kplus = []
images_piplus = []
images_flat_kplus = []
images_flat_piplus = []

times_kplus = []
times_kplus_flat = []
times_piplus = []
times_piplus_flat = []

# initialize images
image_kplus = np.zeros(shape=(48,144,2))
image_piplus = np.zeros(shape=(48,144,2))

num_channels = 2

def get_data(kplus,piplus,num_channels=2,time_cut=50,eventMax=10000,images_kplus=images_kplus,images_piplus=images_piplus,times_kplus=times_kplus,times_piplus=times_piplus):

  eventMax = eventMax  # originially 10k
  eventCounter = 0

  print("Filling image arrays")
  # loop over kplus events
  for (PixelTimes,PixelRows,PixelCols) in zip(kplus.array("PixelTime"),kplus.array("PixelRow"),kplus.array("PixelCol")):
      #print("event")
      image_single = np.zeros(shape=(48,144,num_channels))
      times_single = []
      
      # loop over pixels within event
      for (PixelTime,PixelRow,PixelCol) in zip(PixelTimes,PixelRows,PixelCols):
          #print("PixelTime,Row,Col = %f,%d,%d" % (PixelTime,PixelRow,PixelCol))
          times_single.append(PixelTime)
          if PixelTime < time_cut:
            image_kplus[PixelCol,PixelRow,0] += 1
            image_single[PixelCol,PixelRow,0] = PixelTime
          elif PixelTime >= time_cut:
            image_kplus[PixelCol,PixelRow,1] += 1
            image_single[PixelCol,PixelRow,1] = PixelTime
          # image_single[PixelCol,PixelRow] = PixelTime
          #h2_kplus.Fill(PixelRow,PixelCol)
      
      # after each event
      images_kplus.append(image_single)
      # images_flat_kplus.append(np.reshape(image_single, 6912))

      times_kplus += times_single
      
      eventCounter = eventCounter+1
      if eventCounter > eventMax:
          break

  eventCounter = 0
          
  # loop over piplus events
  for (PixelTimes,PixelRows,PixelCols) in zip(piplus.array("PixelTime"),piplus.array("PixelRow"),piplus.array("PixelCol")):
      #print("event")
      image_single2 = np.zeros(shape=(48,144,num_channels))
      times_single = []

      # loop over pixels within event
      for (PixelTime,PixelRow,PixelCol) in zip(PixelTimes,PixelRows,PixelCols):
          #print("PixelTime,Row,Col = %f,%d,%d" % (PixelTime,PixelRow,PixelCol))
          times_single.append(PixelTime)
          if PixelTime < time_cut:
            image_piplus[PixelCol,PixelRow,0] += 1
            image_single2[PixelCol,PixelRow,0] = PixelTime
          elif PixelTime >= time_cut:
            image_piplus[PixelCol,PixelRow,1] += 1
            image_single2[PixelCol,PixelRow,1] = PixelTime
          # image_single2[PixelCol,PixelRow] = PixelTime
          #h2_piplus.Fill(PixelRow,PixelCol)
      
      # after each event
      ''' ORIGINALLY THIS IS APPENDING THE WRONG IMAGE (IMAGE_SINGLE INSTEAD OF IMAGE_SINGLE2)'''
      images_piplus.append(image_single2)
      # images_flat_piplus.append(np.reshape(image_single, 6912))
      
      times_piplus += times_single

      eventCounter = eventCounter+1
      if eventCounter > eventMax:
          break

# Min and Max Momenta 
minP = 6
maxP = 6
numEvents = 10000

# Fill theta4 np arrays
for i in range(minP-3,maxP-2):
  entry = data[0][i]
  get_data(entry[0],entry[1],num_channels=num_channels,time_cut=50,eventMax=numEvents)

# # Fill theta8 np arrays
# for i in range(minP-3,maxP-2):
#   entry = data[1][i]
#   get_data(entry[0],entry[1],num_channels=num_channels,time_cut=50,eventMax=numEvents)

# # Fill phim45 np arrays
# for i in range(minP-3,maxP-2):
#   entry = data[2][i]
#   get_data(entry[0],entry[1],num_channels=num_channels,time_cut=65,eventMax=numEvents)

print(images_kplus[0].shape)
print(len(times_kplus))
print(np.amax(times_kplus))
print(np.amin(times_kplus))
print(len(times_piplus))
print(np.amax(times_piplus))
print(np.amin(times_piplus))

In [0]:
mom = '6GeV'
# Plot hit times
bins = 200
low = min(np.min(times_kplus),np.min(times_piplus))
high = 200 #np.max(times)
low_high = (low,high)
f = plt.figure()
plt.clf()
plt.hist(times_piplus, color='b', alpha=0.5, range=low_high, bins=bins, histtype='step', density=False, label='Hit Times Pi+')
plt.hist(times_kplus, color='r', alpha=0.5, range=low_high, bins=bins, histtype='step', density=False, label='Hit Times K+')
plt.xlabel('Hit Time (ns)')
plt.ylabel('Number of Hits')
plt.legend(loc='best')
f.savefig('hit_times_'+mom+'.png')
plt.show()

# draw cumulative channel 1 images
f = plt.figure()
plt.subplot(2, 1, 1)
ax1 = plt.gca()
plt.title(mom+" Channel 1 (t<50ns) Hit Pattern K+ (top) and Pi+ (bottom)")
im1 = plt.imshow(image_kplus[:,:,0], cmap='viridis') #, vmin=0, vmax=500)
divider1 = make_axes_locatable(ax1)
cax1 = divider1.append_axes("right", size="5%", pad=0.05)
cb1 = plt.colorbar(im1, cax=cax1)
cb1.set_label("Hits")
cb1.minorticks_on()
plt.subplot(2, 1, 2)
ax2 = plt.gca()
plt.title("")
im2 = plt.imshow(image_piplus[:,:,0], cmap='viridis') #, vmin=0, vmax=500
divider2 = make_axes_locatable(ax2)
cax2 = divider2.append_axes("right", size="5%", pad=0.05)
cb2 = plt.colorbar(im2, cax=cax2)
cb2.set_label("Hits")
cb2.minorticks_on()
plt.show()
f.savefig('kp_pip_ch1_cumulative.png')

# draw cumulative channel 2 images
f = plt.figure()
plt.subplot(2, 1, 1)
ax1 = plt.gca()
plt.title(mom+" Channel 2 (t>=50ns) Hit Pattern K+ (top) and Pi+ (bottom)")
im1 = plt.imshow(image_kplus[:,:,1], cmap='viridis') #, vmin=0, vmax=500
divider1 = make_axes_locatable(ax1)
cax1 = divider1.append_axes("right", size="5%", pad=0.05)
cb1 = plt.colorbar(im1, cax=cax1)
cb1.set_label("Hits")
cb1.minorticks_on()
plt.subplot(2, 1, 2)
ax2 = plt.gca()
plt.title("")
im2 = plt.imshow(image_piplus[:,:,1], cmap='viridis') #, vmin=0, vmax=500
divider2 = make_axes_locatable(ax2)
cax2 = divider2.append_axes("right", size="5%", pad=0.05)
cb2 = plt.colorbar(im2, cax=cax2)
cb2.set_label("Hits")
cb2.minorticks_on()
plt.show()
f.savefig('kp_pip_ch2_cumulative.png')

# draw single channel 1 images
f = plt.figure()
plt.subplot(2, 1, 1)
ax1 = plt.gca()
plt.title(mom+" Channel 1 (t<50ns) Hit Pattern K+ (top) and Pi+ (bottom)")
im1 = plt.imshow(images_kplus[1][:,:,0], cmap='viridis')
divider1 = make_axes_locatable(ax1)
cax1 = divider1.append_axes("right", size="5%", pad=0.05)
cb1 = plt.colorbar(im1, cax=cax1)
cb1.set_label("Hits")
cb1.minorticks_on()
plt.subplot(2, 1, 2)
ax2 = plt.gca()
plt.title("")
im2 = plt.imshow(images_piplus[1][:,:,0], cmap='viridis', vmin=0, vmax=500) #, vmin=0, vmax=500
divider2 = make_axes_locatable(ax2)
cax2 = divider2.append_axes("right", size="5%", pad=0.05)
cb2 = plt.colorbar(im2, cax=cax2)
cb2.set_label("Hits")
cb2.minorticks_on()
plt.show()
f.savefig('kp_pip_ch1_single.png')

# draw single channel 2 images
f = plt.figure()
plt.subplot(2, 1, 1)
ax1 = plt.gca()
plt.title(mom+" Channel 2 (t>=50ns) Hit Pattern K+ (top) and Pi+ (bottom)")
im1 = plt.imshow(images_kplus[1][:,:,1], cmap='viridis')
divider1 = make_axes_locatable(ax1)
cax1 = divider1.append_axes("right", size="5%", pad=0.05)
cb1 = plt.colorbar(im1, cax=cax1)
cb1.set_label("Hits")
cb1.minorticks_on()
plt.subplot(2, 1, 2)
ax2 = plt.gca()
plt.title("")
im2 = plt.imshow(images_piplus[1][:,:,1], cmap='viridis', vmin=0, vmax=500) #, vmin=0, vmax=500
divider2 = make_axes_locatable(ax2)
cax2 = divider2.append_axes("right", size="5%", pad=0.05)
cb2 = plt.colorbar(im2, cax=cax2)
cb2.set_label("Hits")
cb2.minorticks_on()
plt.show()
f.savefig('kp_pip_ch2_single.png')

In [0]:
# Preshuffle data
preShuffle = True
if preShuffle:

  # Preshuffle DIRC Data
  np.random.shuffle(images_kplus)
  np.random.shuffle(images_piplus)
  print(len(images_kplus))
  print(len(images_piplus))

In [0]:
# Split samples into training and testing
print("Splitting sample into training and testing subsets")
split = np.array((80,10,10)) # Percentages into which to split data (% train, % val, % test)
n_kplus = len(images_kplus)
n_piplus = len(images_piplus)
n_total = n_kplus + n_piplus
print(n_kplus / split)

print(n_kplus // split)
print(n_piplus // split)
images_train_kplus = images_kplus[:int(n_kplus * split[0] / 100)]
images_train_piplus = images_piplus[:int(n_piplus * split[0] / 100)]
n_train_kplus = len(images_train_kplus)
n_train_piplus = len(images_train_piplus)
n_train_total = n_train_kplus + n_train_piplus

images_val_kplus = images_kplus[int(n_kplus * split[0] / 100):int(n_piplus * (split[0] + split[1]) / 100)]
images_val_piplus = images_piplus[int(n_piplus * split[0] / 100):int(n_piplus * (split[0] + split[1]) / 100)]
n_val_kplus = len(images_val_kplus)
n_val_piplus = len(images_val_piplus)
n_val_total = n_val_kplus + n_val_piplus

images_test_kplus = images_kplus[int(n_piplus * (split[0] + split[1]) / 100):]
images_test_piplus = images_piplus[int(n_piplus * (split[0] + split[1]) / 100):]
n_test_kplus = len(images_test_kplus)
n_test_piplus = len(images_test_piplus)
n_test_total = n_test_kplus + n_test_piplus

data_train = np.concatenate((images_train_kplus, images_train_piplus))
data_val = np.concatenate((images_val_kplus, images_val_piplus))
data_test = np.concatenate((images_test_kplus, images_test_piplus))
data = np.concatenate((images_kplus, images_piplus))

target_train = np.concatenate((np.ones(n_train_kplus), np.zeros(n_train_piplus)))
target_val = np.concatenate((np.ones(n_val_kplus), np.zeros(n_val_piplus)))
target_test = np.concatenate((np.ones(n_test_kplus), np.zeros(n_test_piplus)))
target = np.concatenate((np.ones(n_kplus), np.zeros(n_piplus)))

print('data_train shape =  '+str(data_train.shape))
print('data shape = '+str(data.shape))
print('target shape = '+str(target.shape))
print('unique elements of target = '+str(np.unique(target)))

print("Training size = %d" % len(data_train))
print("Validation size = %d" % len(data_val))
print("Testing size = %d" % len(data_test))


In [0]:
# Normalize pixel absolute values to be between 0 and 1
data_train = data_train / data.max()
data_val = data_val / data.max()
data_test = data_test / data.max()

print("Data shape: "+str(data_train.shape))
print("Target shape: "+str(target_train.shape))

In [0]:
#####################
# Model Definitions #
# Matthew McEneaney #
# 1/12/20           #
#####################
num_channels = num_channels
num_classes = 2
############################################### TensorFlow Simple CNN BASIC VERSION ###########################################

# CNN takes 3 dimensional tensor input (image height, image width, color channel)
model = models.Sequential()
# Padding='same' significantly improves results
# model.add(layers.Masking(mask_value=0.1, input_shape=(48, 144, 1)))
model.add(layers.Conv2D(16, (3, 3), strides=(1,1), padding='same', activation='relu', input_shape=(48, 144, num_channels)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(16, activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

tf_cnn_basic = model
# print('1')
############################################### TensorFlow Simple CNN ###########################################

# CNN takes 3 dimensional tensor input (image height, image width, color channel)
model = models.Sequential()
# Padding='same' significantly improves results
# model.add(layers.Masking(mask_value=0.1, input_shape=(48, 144, 1)))
model.add(layers.Conv2D(32, (3, 3), strides=(1,1), padding='same', activation='relu', input_shape=(48, 144, num_channels)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
# model.add(layers.Dropout(0.1))
model.add(layers.Dense(16, activation='relu'))
# model.add(layers.Dropout(0.1))
model.add(layers.Dense(num_classes, activation='softmax'))

tf_cnn_simple = model

############################################### TensorFlow Simple CNN with Dropout ###########################################

# CNN takes 3 dimensional tensor input (image height, image width, color channel)
model = models.Sequential()
# Padding='same' significantly improves results
# model.add(layers.Masking(mask_value=0.1, input_shape=(48, 144, 1)))
model.add(layers.Conv2D(32, (3, 3), strides=(1,1), padding='same', activation='relu', input_shape=(48, 144, num_channels)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

tf_cnn_simple_DO05 = model

############################################### TensorFlow Simple CNN with Dropout and L2 Regularization ###########################################

# CNN takes 3 dimensional tensor input (image height, image width, color channel)
model = models.Sequential()
# Padding='same' significantly improves results
# model.add(layers.Masking(mask_value=0.1, input_shape=(48, 144, 1)))
model.add(layers.Conv2D(32, (3, 3), strides=(1,1), padding='same', activation='relu', input_shape=(48, 144, num_channels),kernel_regularizer=regularizers.l2(0.0001)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

tf_cnn_simple_DO05_L210em4 = model

############################################### TensorFlow Example Architecture ###########################################
# check out https://www.tensorflow.org/tutorials/images/cnn
# CNN takes 3 dimensional tensor input (image height, image width, color channel)
model = models.Sequential()
# Padding='same' significantly improves results
# model.add(layers.Masking(mask_value=0.1, input_shape=(48, 144, 1)))
model.add(layers.Conv2D(32, (3, 3), strides=(1,1), padding='same', activation='relu', input_shape=(48, 144, num_channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), strides=(1,1), activation='relu'))
model.add(layers.Flatten())
# model.add(layers.Dropout(0.1))
model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(0.1))
model.add(layers.Dense(num_classes, activation='softmax'))

tf_cnn_example = model

############################################### TensorFlow Example Architecture ###########################################
# check out https://www.tensorflow.org/tutorials/images/cnn
# CNN takes 3 dimensional tensor input (image height, image width, color channel)
model = models.Sequential()
# Padding='same' significantly improves results
# model.add(layers.Masking(mask_value=0.1, input_shape=(48, 144, 1)))
model.add(layers.Conv2D(32, (3, 3), strides=(1,1), padding='same', activation='relu', input_shape=(48, 144, num_channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), strides=(1,1), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

tf_cnn_example_DO05 = model

################################### Simple CNN for CIFAR-10 Dataset, use Adam Optimizer #######################################
# Check out https://keras.io/examples/cifar10_cnn/ for examples of this architecture and others

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                input_shape=(48, 144, num_channels)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

tf_cifar_example = model

# # Example used this optimizer
# # initiate RMSprop optimizer
# opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

####################################### Try Bottleneck Layers ############################################

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                input_shape=(48, 144, num_channels)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (1, 1))) # Bottleneck layers
model.add(Conv2D(64, (3, 3))) # ...
model.add(Conv2D(64, (1, 1))) # ...
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

tf_cnn_bottelneck = model

#################################### Try Factorization Layers #########################################

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                input_shape=(48, 144, num_channels)))
model.add(Activation('relu'))
model.add(Conv2D(64, (1, 3))) # Factorization layers
model.add(Conv2D(64, (3, 1))) # ...
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (1, 3))) # Factorization layers
model.add(Conv2D(64, (3, 1))) # ...
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

tf_cifar_factorization = model

#################################### Try L2 Regularization #########################################

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
          input_shape=(48, 144, num_channels)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, 
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

tf_cifar_regularized = model

#################################### Try L2 Regularization greater value #########################################

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
          input_shape=(48, 144, num_channels)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, 
          kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

tf_cifar_L210em4 = model

#################################### Try L2 Regularization greater value #########################################

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
          input_shape=(48, 144, num_channels)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, 
          kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

tf_cifar_DO05_L210em4 = model

#################################### Try L2 Regularization with more parallel #########################################

model = models.Sequential()
model.add(Conv2D(128, (3, 3), padding='same',
          input_shape=(48, 144, num_channels)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, 
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

tf_cifar_reg_par = model

#################################### Try L2 Regularization with Larger CNN #########################################

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
          input_shape=(48, 144, num_channels)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, 
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

tf_cifar_reg_conv3 = model

#################################### Try L2 Regularization with Larger CNN #########################################

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
          input_shape=(48, 144, num_channels)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, 
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

tf_cifar_reg_conv4 = model

#################################### Try ELU Activation with L2 Regularization #########################################

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
          input_shape=(48, 144, num_channels)))
model.add(Activation('elu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(64, (3, 3),
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, 
          kernel_regularizer=regularizers.l2(0.00001)))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

tf_cifar_regwELU = model
#####################################################################################################################

In [0]:
############################################### TensorFlow NASNetMobile with DIRC input shape ###########################################
# check out https://www.tensorflow.org/tutorials/images/transfer_learning
# from ipynb.fs.full.DIRC_CNN_Architecture_Evaluation import evaluate_architecture
# for 331x331x3 inputs batch = 320 results in Resource Exhausted Error: OOM when allocating tensor with shape... 

base_model = NASNetMobile(input_shape=(48,144,num_channels),
                       include_top=False,
                       weights=None)
base_model.trainable = True
# Unfreeze top layers
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# # Fine-tune from this layer onwards
# fine_tune_at = 400
# Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#   layer.trainable =  False
# base_model.summary()

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
flatten_layer = tf.keras.layers.Flatten()
dense_layer = tf.keras.layers.Dense(1000,activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.5)
prediction_layer = tf.keras.layers.Dense(2)
tf_NASNetMobile_DIRC = tf.keras.Sequential([base_model,global_average_layer,dropout_layer,prediction_layer])


In [0]:
# from tensorflow.python.keras.utils.generic_utils import Progbar
#@keras_export('keras.callbacks.ProgbarLogger')
from tensorflow.python.keras.callbacks import ProgbarLogger
class MetricsLogger(ProgbarLogger):
  """Callback that prints metrics to stdout. 
  
  Modified by Matthew McEneaney, 2/1/2020 10:00:00.
  Arguments:
      count_mode: One of "steps" or "samples".
          Whether the progress bar should
          count samples seen or steps (batches) seen.
      stateful_metrics: Iterable of string names of metrics that
          should *not* be averaged over an epoch.
          Metrics in this list will be logged as-is.
          All others will be averaged over time (e.g. loss, etc).
  Raises:
      ValueError: In case of invalid `count_mode`.
  """

  def __init__(self, logmaster, count_mode='samples', stateful_metrics=None):
    super(MetricsLogger, self).__init__() # ORIGINAL NOT WORKING AS OF 4/1, ATTRIBUTE ERROR WITH PARAMS, SUSPECTED VERSION DISCREPANCY
    # super().__init__()
    if count_mode == 'samples':
      self.use_steps = False
    elif count_mode == 'steps':
      self.use_steps = True
    else:
      raise ValueError('Unknown `count_mode`: ' + str(count_mode))
    self.stateful_metrics = set(stateful_metrics or [])
    self.log_values = None
    self.log_master = logmaster

  def on_train_begin(self, logs=None):
    self.verbose = self.params['verbose']
    self.epochs = self.params['epochs']

  def on_epoch_begin(self, epoch, logs=None):
    self.seen = 0
    if self.use_steps:
      self.target = self.params['steps']
    # else:
      # self.target = self.params['samples']

    # if self.verbose:
    #   if self.epochs > 1:
    #     print('Epoch %d/%d' % (epoch + 1, self.epochs))
    # self.progbar = Progbar(
    #     target=self.target,
    #     verbose=self.verbose,
    #     stateful_metrics=self.stateful_metrics,
    #     unit_name='step' if self.use_steps else 'sample')

  def on_batch_begin(self, batch, logs=None):
    self.log_values = []

  def on_batch_end(self, batch, logs=None):
    logs = logs or {}
    batch_size = logs.get('size', 0)
    # In case of distribution strategy we can potentially run multiple steps
    # at the same time, we should account for that in the `seen` calculation.
    num_steps = logs.get('num_steps', 1)
    if self.use_steps:
      self.seen += num_steps
    else:
      self.seen += batch_size * num_steps

    for k in self.params['metrics']:
      if k in logs:
        self.log_values.append((k, logs[k]))
        self.log_master.append((k,logs[k]))

    # Skip progbar update for the last batch;
    # will be handled by on_epoch_end.
    # if self.verbose and (self.target is None or self.seen < self.target):
    #   self.progbar.update(self.seen, self.log_values)

  def on_epoch_end(self, epoch, logs=None):
    logs = logs or {}
    for k in self.params['metrics']:
      if k in logs:
        self.log_values.append((k, logs[k]))
    # if self.verbose:
    #   self.progbar.update(self.seen, self.log_values)


In [0]:
###################################
# Evaluation function definitions #
# Matthew McEneaney               #
# 1/12/20                         #
####################################

def write_summary(model,name,verbosity):
  '''
  Write model architecture summary into a .txt file.
  '''

  def myFunc(text):
    outF.write(text+'\n')
  
  # Write summary to file
  outF = open(name+"_summary.txt", "w")
  model.summary(print_fn=myFunc)
  outF.close()
  if verbosity == 1:
    model.summary()

def plot_metrics(name,batch_size,results,history,n,verbosity):
  '''
  Plot training metrics (accuracy, validation accuracy, 
  loss, validation, loss) as a function of training 
  epoch and save in .png and .root file formats.
  '''
  chdir('by_epoch')

  # Plot as .png
  f = plt.figure(figsize=(16,10))
  val = plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'],'--', color=val[0].get_color())
  val = plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'],'--', color=val[0].get_color())
  plt.title('Model: '+name+' '+str(n)+' metrics, batch_size = '+str(batch_size)+", test_acc = "+str(results[1]))
  plt.ylabel('Arbitrary Units')
  plt.xlabel('Epoch')
  plt.legend(['accuracy', 'val_accuracy','loss', 'val_loss'], loc='best')
  if verbosity == 1:
    plt.show()
  f.savefig("epochs_metrics"+str(n)+".png")

  chdir('..')

  # Convert accuracy and loss plots into ROOT histograms
  if verbosity == 1:
    print("Converting training epochs plots to ROOT...")

  epochs = np.array([i for i in range(len(history.history['accuracy']))])

  chdir('root_files')

  file = uproot.recreate('epochs_accuracy_'+str(n)+'.root', compression=uproot.ZLIB(4))
  file["acc"] = np.histogram2d(epochs,history.history['accuracy'])

  file = uproot.recreate('epochs_val_accuracy'+str(n)+'.root', compression=uproot.ZLIB(4))
  file["val_acc"] = np.histogram2d(epochs,history.history['val_accuracy'])

  file = uproot.recreate('epochs_loss'+str(n)+'.root', compression=uproot.ZLIB(4))
  file["loss"] = np.histogram2d(epochs,history.history['loss'])

  file = uproot.recreate('epochs_val_loss'+str(n)+'.root', compression=uproot.ZLIB(4))
  file["val_loss"] = np.histogram2d(epochs,history.history['val_loss'],epochs)
  
  chdir('..')

def plot_stability(metrics,name,batch_size,verbosity):
  '''
  Plot the test metrics (accuracy, loss, chi2) recorded for 
  several model iterations in both .png and .root file formats.
  '''

  if verbosity == 1:
    print("Plotting purity and \u03C7\u00b2")

  # Plot accuracy
  f = plt.figure()
  plt.plot(metrics[0],metrics[1],'b.', scaley = False)
  plt.title("Model: "+name+", test acc, batch_size = "+str(batch_size)+", ave = "+str(round(np.average(metrics[1]),4)))
  plt.xlabel("Iteration")
  plt.ylabel("Arbitrary units")
  if verbosity == 1:
    plt.show()
  f.savefig("stability_acc.png")

  # Plot loss
  f = plt.figure()
  plt.plot(metrics[0],metrics[2],'b.', scaley = False)
  plt.title("Model: "+name+", test loss, batch_size = "+str(batch_size)+", ave = "+str(round(np.average(metrics[2]),4)))
  plt.xlabel("Iteration")
  plt.ylabel("Arbitrary units")
  if verbosity == 1:
    plt.show()
  f.savefig("stability_loss.png")

  # Plot kaon chi2
  f = plt.figure()
  plt.plot(metrics[0],metrics[3][0],'b.')
  plt.title("Model: "+name+", Kaon \u03C7\u00b2, batch_size = "+str(batch_size)+", ave = "+str(round(np.average(metrics[3][0]),4)))
  plt.xlabel("Iteration")
  plt.ylabel("Arbitrary units")
  if verbosity == 1:
    plt.show()
  f.savefig("stability_chi2_kaon.png")

  # Plot pion chi2
  f = plt.figure()
  plt.plot(metrics[0],metrics[3][1],'b.')
  plt.title("Model: "+name+", Pion \u03C7\u00b2, batch_size = "+str(batch_size)+", ave = "+str(round(np.average(metrics[3][1]),4)))
  plt.xlabel("Iteration")
  plt.ylabel("Arbitrary units")
  if verbosity == 1:
    plt.show()
  f.savefig("stability_chi2_pion.png")

  # Plot total chi2
  f = plt.figure()
  plt.plot(metrics[0],metrics[3][2],'b.')
  plt.title("Model: "+name+", Total \u03C7\u00b2, batch_size = "+str(batch_size)+", ave = "+str(round(np.average(metrics[3][2]),3)))
  plt.xlabel("Iteration")
  plt.ylabel("Arbitrary units")
  if verbosity == 1:
    plt.show()
  f.savefig("stability_chi2_tot.png")
    
  # Convert accuracy and loss plots into ROOT histograms
  if verbosity == 1:
    print("Converting stability plots to ROOT...")

  chdir('root_files')

  file = uproot.recreate("stability_acc.root", compression=uproot.ZLIB(4))
  file["acc"] = np.histogram2d(metrics[0],metrics[1])

  file = uproot.recreate("stability_loss.root", compression=uproot.ZLIB(4))
  file["loss"] = np.histogram2d(metrics[0],metrics[2])

  file = uproot.recreate("stability_chi2_kaon.root", compression=uproot.ZLIB(4))
  file["chi2k"] = np.histogram2d(metrics[0],metrics[3][0])

  file = uproot.recreate("stability_chi2_pion.root", compression=uproot.ZLIB(4))
  file["chi2p"] = np.histogram2d(metrics[0],metrics[3][1])

  file = uproot.recreate("stability_chi2_tot.root", compression=uproot.ZLIB(4))
  file["chi2t"] = np.histogram2d(metrics[0],metrics[3][2])

  chdir('..')


def evaluate_model(model,metrics,results,n,verbosity):

  '''
  Record the decisions of the CNN (kaon or pion) and 
  test metrics (accuracy and loss) for a single training
  iteration. Also calculate and record the chi2 and error
  for kaon, pion decisions and all decisions respectively.
  '''

  # Keep track of results for stability plots
  metrics[0].append(n)
  metrics[1].append(results[1])
  metrics[2].append(results[0])

  # Evaluate performance by comparing test and training classifier (model) response
  decisions = []
  for (X,y) in ((data_train, target_train), (data_test, target_test)):
      d1 = model.predict_proba(X[y>0.5])[:,1].ravel()
      d2 = model.predict_proba(X[y<0.5])[:,1].ravel()
      decisions += [d1, d2]
      
  print("Filling histograms...")

  bins = 100
  low = min(np.min(d) for d in decisions)
  high = max(np.max(d) for d in decisions)
  low_high = (low,high)
  f = plt.figure()
  plt.clf()
  # Plot training decisions
  plt.hist(decisions[0], color='r', alpha=0.5, range=low_high, bins=bins, histtype='stepfilled', density=True, label='Kaon (train)')
  plt.hist(decisions[1], color='b', alpha=0.5, range=low_high, bins=bins, histtype='stepfilled', density=True, label='Pion (train)')

  # make histogram and get error bars
  hist, bins = np.histogram(decisions[2], bins=bins, range=low_high, density=True)
  width = (bins[1] - bins[0])
  center = (bins[:-1] + bins[1:]) / 2

  # plot histogram for signal test sample (Kaon Test)
  scale = len(decisions[2]) / sum(hist)
  err = np.sqrt(hist * scale) / scale
  plt.errorbar(center, hist, yerr=err, fmt='o', c='r', label='Kaon (test)')

  # make and plot histogram for background test sample (Pion Test)
  hist, bins = np.histogram(decisions[3], bins=bins, range=low_high, density=True)
  scale = len(decisions[2]) / sum(hist)
  err = np.sqrt(hist * scale) / scale
  plt.errorbar(center, hist, yerr=err, fmt='o', c='b', label='Pion (test)')

  plt.xlabel('Classifier output')
  plt.ylabel('Arbitrary units')
  plt.legend(loc='best')
  f.savefig('classifier_'+str(n)+'_output.png')
  if verbosity == 1:
    plt.show()

  # Create test and train decisions histograms for kaon
  hist_kaon_test, bins = np.histogram( decisions[0], bins=bins, range=low_high, density=False)
  hist_kaon_train, bins = np.histogram( decisions[2], bins=bins, range=low_high, density=False)
  
  # Find chi2 values and error for kaon
  hist_kaon_chi2 = np.nan_to_num(np.divide(np.square(np.subtract(hist_kaon_test, hist_kaon_train)),np.add(hist_kaon_test,hist_kaon_train)))
  scale = len(decisions[2]) / sum(hist_kaon_chi2)
  err_kaon = np.sqrt(hist_kaon_chi2 * scale) / scale
  
  # Create test and train decisions histograms for pion
  hist_pion_test, bins = np.histogram( decisions[1], bins=bins, range=low_high, density=False)
  hist_pion_train, bins = np.histogram( decisions[3], bins=bins, range=low_high, density=False)
  
  # Find chi2 values and error for pion
  hist_pion_chi2 = np.nan_to_num(np.divide(np.square(np.subtract(hist_pion_test, hist_pion_train)),np.add(hist_pion_test,hist_pion_train)))
  scale = len(decisions[2]) / sum(hist_pion_chi2)
  err_pion = np.sqrt(hist_pion_chi2 * scale) / scale
  
  # Find chi2 for kaon and pion and total
  chi2_kaon = sum(hist_kaon_chi2)
  chi2_pion = sum(hist_pion_chi2)
  chi2_total = chi2_kaon + chi2_pion
  err_total = math.sqrt(sum(err_kaon**2 + err_pion**2))

  # Record metrics values
  metrics[3][0].append(chi2_kaon)
  metrics[3][1].append(chi2_pion)
  metrics[3][2].append(chi2_total)
  metrics[4][0].append(err_kaon)
  metrics[4][1].append(err_pion)
  metrics[4][2].append(err_total)


def batch_plot(name,batch_size,log_master,n,verbosity):
    losses = []
    accs = []
    for k in log_master:
      if k[0]=='loss':
        losses.append(k[1])
        if len(k) > 2 and verbosity == 1:
          print(len(k))

      elif k[0] == 'accuracy':
        accs.append(k[1])
        if len(k) > 2 and verbosity == 1:
          print(len(k))

    chdir('by_batch')

    f = plt.figure()
    plt.plot(losses)
    plt.title("Model: "+name+str(n)+", Loss vs. batch, batch_size = "+str(batch_size))
    plt.xlabel('batch')
    plt.ylabel('loss')
    if verbosity==1:
      plt.show()
    f.savefig('batches_loss_'+str(n)+'.png')

    f = plt.figure()
    plt.plot(accs)
    plt.title("Model: "+name+str(n)+", Accuracy vs. batch, batch_size = "+str(batch_size))
    plt.xlabel('batch')
    plt.ylabel('accuracy')
    if verbosity==1:
      plt.show()
    f.savefig('batches_acc_'+str(n)+'.png')

    chdir('..')

    # Convert accuracy and loss plots into ROOT histograms
    if verbosity == 1:
      print("Converting batch metrics plots to ROOT...")

    chdir('root_files')

    file = uproot.recreate('batches_loss_'+str(n)+'.root', compression=uproot.ZLIB(4))
    file["losses"] = np.histogram2d([i+1 for i in range(len(losses))],losses)

    file = uproot.recreate('batches_acc_'+str(n)+'.root', compression=uproot.ZLIB(4))
    file["losses"] = np.histogram2d([i+1 for i in range(len(accs))],accs)

    chdir('..')

def evaluate_architecture(architecture,name='',num_classes=2,opt=Adam(),batch=320,epochs=10,iterations=10,verbosity=1):

  '''
  Create a designated folder in which to place all the files for the evaluation of a given architecture.
  Record the architecture of the given architecture and plot and record the training metrics (accuracy, 
  validation accuracy, loss, validation loss) as a function of the training epoch for the given number 
  of epochs (default 10) with a given batch size (default 320).  Also, evaluate the stability of the 
  architecture test metrics (accuracy and loss) over a given number of model iterations (default 10).

  Matthew McEneaney 3/24/20
  '''

  # Internal variable definitions
  metrics = [ [], [], [],  [ [], [], [] ],  [ [], [], [] ]  ]
  minimum = 1
  maximum = iterations + 1
  step = 1
  executable = True
  model = architecture

  # # Create file directory
  for entry in listdir():
    if entry == name:
      while True:
        resp = str(input('The directory ' + name + ' already exists.  Do you wish to overwrite? (y/n):'))
        if resp == 'y':
          rmtree(name)
          executable = True
          break
        elif resp == 'n':
          executable = False
          break
      break
    
  if executable == True:
    # Make file directory
    mkdir(name + '_batchsize' + str(batch))
    chdir(name + '_batchsize' + str(batch))
    mkdir('root_files')
    mkdir('by_epoch')
    mkdir('by_batch')
    mkdir('Model_Checkpoints')

    if name == '':
      print('No model name specified!')
      name = str(input('Please enter name of desired model: '))

    # Record Architecture
    model = tf.keras.models.clone_model(model)
    write_summary(model,name,verbosity)

    # Train architecture several times to assess stability
    for n in range(minimum,maximum,step):
      if verbosity == 1:
        print('#######################\n# Creating model '+str((n - minimum + 1))+'... #\n#######################')
      model = tf.keras.models.clone_model(architecture)
 
      # Learning Rate Schedule, added 1/28/20 9:20:00
      lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(0.001, decay_steps=1000, decay_rate=1) # , decay_steps=STEPS_PER_EPOCH*1000, decay_rate=1, staircase=False

      model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=tf.keras.optimizers.Adam(amsgrad=True), # learning_rate=0.001 decreasing helps eliminate convergence issues but also decreases final quality
                    metrics=['accuracy'])

      # Callback added 1/24/20, check out https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau
      reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                               factor=0.1, patience=10, min_lr=0.001,
                                                               verbose=1, cooldown=10)
      
      # Callback added 1/30/20
      log_dir="logs/fit/" + 'mymodel' + str(n) # datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
      tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

      # Callback added 2/1/20 23:20:00 # no longer working as of 4/1, see callback definition above
      log_master = []
      m_logger = MetricsLogger(log_master)
      
      # Callback added 1/26/20
      early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,verbose=1,baseline=0.01) #restore_best_weights=True

      # Callback added 4/3/20
      checkpoint = ModelCheckpoint('Model_Checkpoints', monitor='val_loss', verbose=0, save_best_only=True, mode='min')

      history = model.fit(data_train,
                          target_train,
                          batch_size=batch,
                          epochs=epochs,
                          verbose=verbosity,
                          callbacks=[tensorboard_callback], #,m_logger, checkpoint this one takes up a lot of RAM?
                          validation_data=(data_val,target_val)) # Error on this line?
      
      results = model.evaluate(data_test,
                               target_test,
                               batch_size=batch,
                               verbose=verbosity)
      
      # Plot trainging metrics as a function of epoch
      plot_metrics(name,batch,results,history,n,verbosity)

      # Evaluate test accuracy, loss, and chi2 for model iteration
      evaluate_model(model,metrics,results,n,verbosity)

      # Plot loss and accuracy metrics as a function of batch
      batch_plot(name,batch,log_master,n,verbosity)

      # Save model
      model.save('model'+str(n))

    # Plot test accuracy, loss, and chi2 as a function of model iteration
    plot_stability(metrics,name,batch,verbosity)

    # Go back up to home directory
    if verbosity == 1:
      listdir()
    chdir("..")
    if verbosity == 1:
      getcwd()
      print("\nDone\n")


In [0]:
# chdir('..')
print(getcwd())
# chdir('..')
# chdir('/content/gdrive/My Drive/DIRC_CNN/test/TPU_Test/')
# rmtree('tf_cifar_regularized_batchsize320')
# mkdir('batchsize32')
# chdir('batchsize32')
print(listdir())
# print(getcwd())
# rmtree('tf_cifar_DO05_L210em4_batchsize320')
# rmtree('tf_cifar_L210em4_batchsize320')
# rmtree('tf_cifar_regularized_batchsize320')


In [0]:
num_channels=2

In [0]:
# myModels = [[tf_cifar_regularized(),'tf_cifar_regularized'],
#             [tf_cnn_example(),'tf_cnn_example'],
#             [tf_cifar_example(),'tf_cifar_example'],
#             [tf_cifar_factorization(),'tf_cifar_factorization'],
#             [tf_cifar_regwELU(),'tf_cifar_regwELU']]
# myModels = [[tf_cnn_basic,'tf_cnn_basic'],
#             [tf_cnn_simple_wDO,'tf_simple_wDO'],
#             [tf_cnn_simple_wDOandL2,'tf_cnn_simple_wDOandL2']]
# myModels = [[tf_cifar_reg_conv3,'tf_cifar_reg_conv3'],
#             [tf_cifar_reg_conv4,'tf_cifar_reg_conv4']]
# myModels = [[tf_cifar_reg_g,'tf_cifar_reg_g'],
#             [tf_cifar_reg_par,'tf_cifar_reg_par']]
# myModels = [[tf_cifar_reg_g,'tf_cifar_reg_g'],
#             [tf_cifar_example,'tf_cifar_example']]
# myModels = [[tf_cifar_regularized,'tf_cifar_regularized']]
myModels = [[tf_cifar_DO05_L210em4,'tf_cifar_DO05_L210em4'],
            [tf_cifar_L210em4,'tf_cifar_L210em4'],
            [tf_cifar_regularized,'tf_cifar_regularized'],
            [tf_cifar_example,'tf_cifar_example'],
            [tf_cnn_example_DO05,'tf_cnn_example_DO05'],
            [tf_cnn_example,'tf_cnn_example'],
            [tf_cnn_simple_DO05_L210em4,'tf_cnn_simple_DO05_L210em4'],
            [tf_cnn_simple_DO05,'tf_cnn_simple_DO05']]
# [tf_NASNetMobile_DIRC,'tf_NASNetMobile_DIRC_2channel']
for entry in myModels:
  evaluate_architecture(entry[0],entry[1],epochs=20,batch=320,iterations=10)
getcwd()
print('Do you wish to continue writing to this directory?')

In [0]:
%tensorboard --logdir logs/fit

In [0]:
# Transfer data to appropriately shaped np arrays
images_kplus = []
images_piplus = []
images_flat_kplus = []
images_flat_piplus = []

times_kplus = []
times_kplus_flat = []
times_piplus = []
times_piplus_flat = []

# initialize images
image_kplus = np.zeros(shape=(48,144,2))
image_piplus = np.zeros(shape=(48,144,2))

get_data(data[0][0],data[0][1],50,10000) # currently theta 4 phim 90 split at 50ns limit is 10000 events

print(images_kplus[0].shape)
print(len(times_kplus))
print(np.amax(times_kplus))
print(np.amin(times_kplus))
print(len(times_piplus))
print(np.amax(times_piplus))
print(np.amin(times_piplus))

In [0]:
# Assimilate test data
print("Splitting sample into training and testing subsets")
n_kplus = len(images_kplus)
n_piplus = len(images_piplus)
n_total = n_kplus + n_piplus

data = np.concatenate((images_kplus, images_piplus))
data_test = data
target = np.concatenate((np.ones(n_kplus), np.zeros(n_piplus)))
target_test = target

print('data shape = '+str(data.shape))
print('target shape = '+str(target.shape))
print('unique elements of target = '+str(np.unique(target)))

print("Testing size = %d" % len(data_test))


In [0]:
# Normalize pixel absolute values to be between 0 and 1
data_test = data_test / data.max()

print("Data shape: "+str(data_test.shape))
print("Target shape: "+str(target_test.shape))

In [0]:
model = tf.keras.models.load_model('/content/gdrive/My Drive/DIRC_CNN/test/Data_Split_Test/tf_cifar_regularized_batchsize320/')
results = model.evaluate(data_test,
                               target_test,
                               batch_size=batch,
                               verbose=verbosity)